In [1]:
import os
import math
import numpy as np
import pandas as pd
from ast import literal_eval
from scipy.special import kl_div
import pymc3
import itertools
import arviz as az
import scipy.stats as st
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as  mpatches
pd.options.mode.chained_assignment = None  # default='warn'

from helper_functions import pre_process_input_data, pre_process_eye_data, point_estimate, pixel_to_degree, get_dist_to_spaceship_fix_rest, get_dist_to_obstacles_fix_explore, get_dist_to_obstacles_sacc

In [2]:
root_dir = os.getcwd()

data_dir = "/experimental_data/"

target_string = "output"
target_string_eye_tracking = "eye_tracking"
done_string = "done"

successfull_runs = []

for subdir, dirs, files in os.walk(root_dir+data_dir):
    for file in files:
        if done_string in file:
            successfull_runs.append(file)

## Picking exemplary data

In [3]:
successfull_runs[0]

'OK01UE_output_5FN_done_10.csv'

In [4]:
code = successfull_runs[0][:6]
triplet = successfull_runs[0][14:17]
run = successfull_runs[0][23:25]

file_name = f'{code}_eye_tracking_output_{triplet}_{run}.csv'

data = pre_process_eye_data(pd.read_csv((f"{root_dir + data_dir}{code}/eye_data/{file_name}"), index_col=False))
data

,TimeTag,LeftEyeX,LeftEyeY,LeftPupilDiameter,RightEyeX,RightEyeY,RightPupilDiameter,DigitalIn,LeftBlink,RightBlink,...,distance_to_spaceship_in_pixel,distance_to_spaceship,exploring_fixation,Saccade,saccadeOnset,N_saccade,saccade_direction_x,saccade_direction_y,saccade_amplitude_in_pixel,saccade_amplitude
0,5011.7430,NaN,NaN,9999.0,NaN,NaN,9999.0,16777215.0,1.0,1.0,...,NaN,NaN,0,0.0,0,NaN,NaN,NaN,NaN,NaN
1,5011.7435,NaN,NaN,9999.0,NaN,NaN,9999.0,16777215.0,1.0,1.0,...,NaN,NaN,0,0.0,0,NaN,NaN,NaN,NaN,NaN
2,5011.7440,NaN,NaN,9999.0,NaN,NaN,9999.0,16777215.0,1.0,1.0,...,NaN,NaN,0,0.0,0,NaN,NaN,NaN,NaN,NaN
3,5011.7445,NaN,NaN,9999.0,NaN,NaN,9999.0,16777215.0,1.0,1.0,...,NaN,NaN,0,0.0,0,NaN,NaN,NaN,NaN,NaN
4,5011.7450,NaN,NaN,9999.0,NaN,NaN,9999.0,16777215.0,1.0,1.0,...,NaN,NaN,0,0.0,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58762,5041.1240,NaN,NaN,9999.0,NaN,NaN,9999.0,16777215.0,1.0,1.0,...,NaN,NaN,0,0.0,0,NaN,NaN,NaN,NaN,NaN
58763,5041.1245,NaN,NaN,9999.0,NaN,NaN,9999.0,16777215.0,1.0,1.0,...,NaN,NaN,0,0.0,0,NaN,NaN,NaN,NaN,NaN
58764,5041.1250,NaN,NaN,9999.0,NaN,NaN,9999.0,16777215.0,1.0,1.0,...,NaN,NaN,0,0.0,0,NaN,NaN,NaN,NaN,NaN
58765,5041.1255,NaN,NaN,9999.0,NaN,NaN,9999.0,16777215.0,1.0,1.0,...,NaN,NaN,0,0.0,0,NaN,NaN,NaN,NaN,NaN


In [5]:
data.columns

Index(['TimeTag', 'LeftEyeX', 'LeftEyeY', 'LeftPupilDiameter', 'RightEyeX',
       'RightEyeY', 'RightPupilDiameter', 'DigitalIn', 'LeftBlink',
       'RightBlink', 'DigitalOut', 'LeftEyeFixationFlag',
       'RightEyeFixationFlag', 'LeftEyeSaccadeFlag', 'RightEyeSaccadeFlag',
       'MessageCode', 'LeftEyeRawX', 'LeftEyeRawY', 'RightEyeRawX',
       'RightEyeRawY', 'time_tag', 'Fixation', 'fixationOnset', 'N_fixation',
       'fixation_duration', 'converging_eye_x', 'converging_eye_y',
       'converging_eye_x_adjusted', 'converging_eye_y_adjusted',
       'distance_to_spaceship_in_pixel', 'distance_to_spaceship',
       'exploring_fixation', 'Saccade', 'saccadeOnset', 'N_saccade',
       'saccade_direction_x', 'saccade_direction_y',
       'saccade_amplitude_in_pixel', 'saccade_amplitude'],
      dtype='object')

# Identifying action goals

We hypothesize that action goals are strongly linked to visual attention. Therefore when the visual attention stays at the same relative position within the instance, this might be the action goal - the position where the spaceship is actively steered towards.

Oculomotor control however combined top-down (action goal) and bottom-up (stimulus driven) processes. We have to identify action goals even when the eyes move somewhere else. Important is, that the eyes fixated on the action goal initially and the spaceship got closer and closer over time.

Action goals might therefore be accompanied by a progressive (downwards) saccade and a short period of smooth pursuit (fixation at relative position) converging with the spaceship.

In [6]:
# condition for saccade landing site (last saccade frame)
cond = (data.Saccade >= 1.0) & (data.Saccade.shift(-1) == 0.0)

# have =1 everywhere condition applies and =0 where not
data["saccadeland"] = np.where(cond, 1, 0)

In [7]:
sacclands = data[data["saccadeland"] == 1]

Now we have the lines in which the saccade landed. We want to identify smooth pursuits after these saccades. Smooth pursuits should usually be marked as fixations. The slow travel of the eyes during these frames is still below the threshold and therefore they are flagged as Fixation==1 by the TRACKPixx algorithm. Consecutively, we will split the data into chunks from the saccadeland until another saccade is initiated. We will further explore the movement in the eyes within those chunks.

In [25]:
mean_y_velocities = []
sd_y_velocities = []

for row_id, row in sacclands.iterrows():
    index = row_id+1
    # cut off all data in front
    tail = data.iloc[index+1:]
    # identify end index
    try:        
        # when a saccade is initiated, that's when our chunk is sliced off
        end_index = tail[(tail.Saccade == 1)].index[0]
        sliced_data = data.iloc[index:end_index]
    except IndexError:
        # else until the end because we will apply nanmean therefore do not fear having all nans of the end in the chunk
        sliced_data = data.iloc[index:]
    
    steps_y = [i-j for i, j in zip(sliced_data.converging_eye_y_adjusted[:-1], sliced_data.converging_eye_y_adjusted[1:])]
    
    # plot KDE of steps_y
        
    #print(np.nanmean(steps_y))
    mean_y_velocities.append(np.nanmean(steps_y))
    sd_y_velocities.append(np.nanstd(steps_y))
    

482
1621
444
1128
434
1062
669
782
1041
3079
3172
417
2077
336
2652
4624
548
559
690
1761
859
556
558
3060
2693
1197
2954
1082
1020
1738
654
1688
511
240
3801
845
1711


/var/folders/z1/crk0j_p52yq850yynjpq93wm0000gn/T/ipykernel_1255/1877698351.py:26: RuntimeWarning: Mean of empty slice
  mean_y_velocities.append(np.nanmean(steps_y))


In [22]:
mean_y_velocities

[-0.2769093979421128,
 -0.08397701615137411,
 0.3407511485080418,
 0.015796732458268632,
 0.26889792215466224,
 0.005829576497702639,
 -0.20200471249883045,
 0.3950110359927061,
 -0.07998117300180288,
 0.055996563312215725,
 0.055178976404821035,
 -0.4167445256159856,
 -0.058865826951870315,
 -0.2908900929920709,
 0.08259000601924982,
 0.026661595653136427,
 -0.36440662026765486,
 -0.23162442552573365,
 -0.22491247909187337,
 0.09405195062810724,
 0.09886265467930506,
 0.28104583465301236,
 -0.13360738155015708,
 0.06331346953916721,
 0.06893864216457327,
 0.05790394046234845,
 0.04952869615512826,
 0.1317830502408222,
 -0.09543107232120952,
 0.0391852028011658,
 0.4537811279296875,
 -0.08735028115969547,
 0.3176252178117341,
 -0.39100423517586297,
 -0.02496580332388011,
 -0.4934784602929299,
 nan]

### Finding out velocity in pixel for game visualization in y direction

In [11]:
input_file_name = "OK01UE_output_5FN_done_10.csv"
button_data = pre_process_input_data(pd.read_csv((f"{root_dir + data_dir}{code}/data/{input_file_name}"), index_col=False))

button_data[button_data['visible_obstacles'].str.len() > 0].visible_obstacles

197     [[684, 810], [828, 810]]
198     [[684, 804], [828, 804]]
199     [[684, 798], [828, 798]]
200     [[684, 792], [828, 792]]
201     [[684, 786], [828, 786]]
                  ...           
1423                 [[888, 24]]
1424                 [[888, 18]]
1425                 [[888, 12]]
1426                  [[888, 6]]
1427                  [[888, 0]]
Name: visible_obstacles, Length: 1231, dtype: object

Comparing the first two lines which are consecutive game frames, we can see that the y variable of the first obstacles is reduced by 6 (810 -> 804). That is our ingame velocity.

next the ingame velocity for eye-movement data:

In [12]:
factor = 2000 / 60
y_vel = 6/factor
y_vel

0.18